In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def true_range(data):
    high_minus_low = data['High'] - data['Low']
    high_minus_prev_close = np.abs(data['High'] - data['Close'].shift())
    low_minus_prev_close = np.abs(data['Low'] - data['Close'].shift())
    return np.max([high_minus_low, high_minus_prev_close, low_minus_prev_close], axis=0)

def media(data, periodo):
    # Calculate the simple moving average using pandas rolling window
    return data['TrueRange'].rolling(periodo).mean()

def calculate_satrc_satrv(data, periodo, desvio):
    satrc = [0]
    satrv = [0]

    for i in range(1, len(data)):
        ATR = media(data, periodo)[i]

        SATRc1 = abs(data['Close'].iloc[i] - (ATR * desvio))
        SATRv1 = abs(data['Close'].iloc[i] + (ATR * desvio))

        if data['Close'].iloc[i] >= satrc[-1]:
            if SATRc1 > satrc[-1]:
                satrc.append(SATRc1)
            else:
                satrc.append(satrc[-1])
        else:
            satrc.append(SATRc1)

        if data['Close'].iloc[i] < satrv[-1]:
            if SATRv1 < satrv[-1]:
                satrv.append(SATRv1)
            else:
                satrv.append(satrv[-1])
        else:
            satrv.append(SATRv1)

    # Ensure that the lengths of satrc and satrv match data.index
    satrc = satrc[:-1]
    satrv = satrv[:-1]

    return satrc, satrv

# Example usage with Ibovespa data
# Replace 'ibov_data.csv' with the path to your actual Ibovespa data file
data = df

# Calculate the True Range and add it to the DataFrame
data['TrueRange'] = true_range(data)

periodo = 20
desvio = 2.00

satrc, satrv = calculate_satrc_satrv(data, periodo, desvio)

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(data.index[1:], satrc, label='SATRc')
plt.plot(data.index[1:], satrv, label='SATRv')
plt.plot(data.index, data['Close'].shift(1), label='Close', color='black', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Indicator Values')
plt.title('Stop ATR Indicator')
plt.legend()
plt.grid(True)
plt.show()
